In [ ]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [3]:
import minsearch

In [4]:
import json

In [5]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [6]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [37]:
# documents[0]

In [76]:
index = minsearch.Index(
    text_fields = ["question","text","section"],
    keyword_fields = ["course"]
)

In [77]:
index.fit(documents)

In [78]:
from openai import OpenAI

In [79]:
client = OpenAI()

In [80]:
def search(query):
    boost = {'question': 3.0, 'section':0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=10
    )
    return results

In [86]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database. 
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    
    context = ""

    for doc in search_results:
        context = context + f"section:  {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n" 

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [87]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user","content":prompt}]
    )
    return response.choices[0].message.content

In [88]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [89]:
rag('On this course, how do I run Kafka?')

'To run Kafka in this course, you should follow these steps:\n\n1. **Create a Virtual Environment (Recommended for Running Python Files):** Create a virtual environment to run the Python files for Kafka.\n\n   ```bash\n   python -m venv env\n   source env/bin/activate  # On MacOS/Linux\n   # or\n   env\\Scripts\\activate  # On Windows\n   ```\n\n2. **Install Required Packages:**\n\n   ```bash\n   pip install -r ../requirements.txt\n   ```\n\n3. **Activate the Virtual Environment Every Time:**\n\n   ```bash\n   source env/bin/activate  # On MacOS/Linux\n   # or\n   env\\Scripts\\activate  # On Windows\n   ```\n\n4. **Java Kafka Setup:** For running Java-based Kafka producer/consumer/etc., navigate to the project directory and use the following command:\n\n   ```bash\n   java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n   ```\n\n5. **Docker Containers:** Ensure that Docker images are up and running, as the virtual environment is meant only 

In [90]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [91]:
from elasticsearch import Elasticsearch

In [92]:
es_client = Elasticsearch('http://localhost:9200')

In [93]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [94]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [95]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:24<00:00, 38.22it/s]


In [113]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    return result_docs

In [114]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [115]:
rag(query)

"To run Kafka, you can follow these instructions from our course module on streaming with Kafka:\n\nIf you want to run a Java Kafka producer or other components (like consumer or kstreams) in the terminal, you should navigate to your project directory and use the following command:\n\n```shell\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nReplace `<jar_name>` with the actual name of your jar file. Ensure you've built the project and have the necessary jar files ready for execution."